In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf

# Function to scrape S&P 500 symbols
def scrape_sp500_symbols(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    symbols = []
    table = soup.find('table', {'class': 'wikitable'})

    if table:
        rows = table.find_all('tr')[1:]
        for row in rows:
            cols = row.find_all('td')
            if len(cols) > 0:
                symbol = cols[0].text.strip()
                name = cols[1].text.strip()
                symbols.append((symbol, name))

    symbols_df = pd.DataFrame(symbols, columns=['Symbol', 'Name'])
    symbols_df['ID'] = range(1, len(symbols_df) + 1)
    symbols_df = symbols_df[['ID', 'Symbol', 'Name']]

    return symbols_df

# Scraping S&P 500 symbols
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
symbols_df = scrape_sp500_symbols(url)

# Function to get historical prices for all symbols
def get_historical_prices(symbols_df, start_date, end_date):
    historical_data = []

    for index, row in symbols_df.iterrows():
        symbol = row['Symbol']
        try:
            # Fetch historical prices
            stock_data = yf.download(symbol, start=start_date, end=end_date)
            stock_data.reset_index(inplace=True)

            for _, price_row in stock_data.iterrows():
                historical_data.append({
                    'ID': row['ID'],
                    'Symbol': symbol,
                    'Date': price_row['Date'],
                    'Open': price_row['Open'],
                    'High': price_row['High'],
                    'Low': price_row['Low'],
                    'Close': price_row['Close'],
                    'Volume': price_row['Volume']
                })
        except Exception as e:
            print(f"Error retrieving data for {symbol}: {e}")

    return pd.DataFrame(historical_data)

# Define the date range
start_date = '2018-01-01'
end_date = '2024-01-01'

# Get historical prices DataFrame
historical_prices_df = get_historical_prices(symbols_df, start_date, end_date)

# Function to calculate metrics with moving averages
def calculate_metrics(historical_prices_df):
    metrics_data = []

    # Calculate moving averages (1 to 50 days) for each stock symbol
    for (stock_id, symbol), group in historical_prices_df.groupby(['ID', 'Symbol']):
        for i in range(1, 51):
            group[f'Moving-Avg-{i}'] = group['Close'].rolling(window=i).mean()

        # Store the required columns for calculated_metrics_df
        for _, row in group.iterrows():
            metrics_data.append({
                'ID': stock_id,
                'Symbol': symbol,
                'Date': row['Date'],
                'Close': row['Close'],
                **{f'Moving-Avg-{i}': row[f'Moving-Avg-{i}'] for i in range(1, 51)}
            })

    return pd.DataFrame(metrics_data)

# Create calculated_metrics_df
calculated_metrics_df = calculate_metrics(historical_prices_df)

# Display the first few rows of each DataFrame
print("Symbols DataFrame:")
print(symbols_df.head())
print("\nHistorical Prices DataFrame:")
print(historical_prices_df.head())
print("\nCalculated Metrics DataFrame:")
print(calculated_metrics_df.head())

# Save DataFrames to CSV
symbols_df.to_csv('sp500_symbols.csv', index=False)
historical_prices_df.to_csv('historical_prices.csv', index=False)
calculated_metrics_df.to_csv('calculated_metrics.csv', index=False)

print("\nCSV files created successfully!")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Symbols DataFrame:
   ID Symbol                 Name
0   1    MMM                   3M
1   2    AOS          A. O. Smith
2   3    ABT  Abbott Laboratories
3   4   ABBV               AbbVie
4   5    ACN            Accenture

Historical Prices DataFrame:
   ID Symbol       Date        Open        High         Low       Close  \
0   1    MMM 2018-01-02  197.140472  198.219070  194.657196  197.023407   
1   1    MMM 2018-01-03  196.546829  197.098663  195.058533  197.015045   
2   1    MMM 2018-01-04  198.160538  200.200668  197.717392  199.590302   
3   1    MMM 2018-01-05  199.540131  201.421402  198.779266  201.145493   
4   1    MMM 2018-01-08  200.150497  201.454849  199.983276  200.493317   

    Volume  
0  3505476  
1  2623665  
2  2682748  
3  2195736  
4  2235324  

Calculated Metrics DataFrame:
   ID Symbol       Date       Close  Moving-Avg-1  Moving-Avg-2  Moving-Avg-3  \
0   1    MMM 2018-01-02  197.023407    197.023407           NaN           NaN   
1   1    MMM 2018-01-03  